In [106]:
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry

In [107]:
from enum import IntEnum

class LV_SURFS(IntEnum):
  ENDO = 0
  EPI = 1
  ATRIAL = 2
  MITRAL = 3
  INTERCECTION = 4

In [108]:
# jupyter_backend='pythreejs'
pv.set_jupyter_backend("pythreejs")

In [109]:
FILE_PATH = "C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models/Full_Heart_Mesh_1.vtk"

In [110]:
lv = Geometry()
lv.from_pyvista_read(FILE_PATH, identifier="elemTag", threshold=[0, 1])

In [111]:
lvsurf = lv.mesh.extract_surface()
lvsurf.compute_normals(inplace=True)

PolyData (0x16bc94772e0)
  N Cells:	104390
  N Points:	52193
  X Bounds:	-2.362e+01, 6.196e+01
  Y Bounds:	-4.083e+01, 4.766e+01
  Z Bounds:	-5.273e+01, 3.977e+01
  N Arrays:	5

In [112]:
normals = lvsurf.get_array("Normals", "points")
normals.shape

(52193, 3)

In [113]:
center = np.mean(lvsurf.points, axis=0)
center

pyvista_ndarray([22.56839688,  0.14375042, -6.93471445])

In [114]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

In [115]:
def angle_between(v1, v2, check_orientation=True, zaxis=[0.,0.,1.]):
    """ 
      Returns the angle in radians between vectors 'v1' and 'v2'
    """
    #  compute angle
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    # if check_orientation:
    #   # make sure angle is in range [0, 2*pi)
    #   zaxis = np.asarray(zaxis)
    #   det = np.linalg.det(np.vstack((v1_u.T, v2_u.T, zaxis.T))) # https://bit.ly/3nUrr0U
    #   if det < 0:
    #     angle = 2*np.pi - angle
    return angle

In [116]:
n_t_c = center - lvsurf.points

angles = np.zeros(len(n_t_c))
for i, (pt_normal, pt_vec) in enumerate(zip(normals, n_t_c)):
  angles[i] = angle_between(pt_vec, pt_normal, check_orientation=False)

# angles = np.degrees(angles)
# lvsurf.point_data["angles"] = angles
# lvsurf.set_active_scalars("angles")

In [89]:
angles.max()

3.141081162118487

In [117]:
angles_tresh = np.copy(angles)
thresh_val = np.radians(90)

angles_tresh[angles<thresh_val]=0
angles_tresh[angles>=thresh_val]=1

angles_tresh[angles>=np.radians(180)]=0

# angles_tresh[angles<np.radians(10)]=1

lvsurf.point_data["angles_tresh"] = angles_tresh
lvsurf.set_active_scalars("angles_tresh")

In [118]:
endo_epi_guess=np.copy(angles_tresh)

In [119]:
plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False)
plotter.add_mesh(lvsurf, scalars="angles_tresh", opacity=1.0, show_edges=False)
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [93]:
lvsurf = lvsurf.compute_derivative("angles_tresh")
lvsurf = lvsurf.compute_derivative("gradient")
lvsurf = lvsurf.compute_derivative("gradient")

grads = lvsurf.get_array("gradient") #/ 180
grads_mag = np.linalg.norm(grads, axis=1)
goi = np.copy(grads_mag)
goi[grads_mag>0] = 1
goi[grads_mag<=0] = 0

In [94]:
lvsurf.point_data["goi"] = goi
lvsurf.set_active_scalars("goi")

In [95]:
plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False)
plotter.add_mesh(lvsurf, scalars="goi", opacity=1.0, show_edges=False)
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [96]:
pts = lvsurf.points
ioi = np.where(goi > 0)[0]
poi = pts[ioi]

In [97]:
from sklearn.cluster import KMeans

n_centroids = 2

kmeans = KMeans(n_clusters=n_centroids, random_state=0).fit(poi)
klabels = kmeans.labels_
kcenters = kmeans.cluster_centers_

In [98]:
kdist = np.linalg.norm(center - kcenters, axis=1)
label = np.zeros(len(klabels))
if kdist[0] < kdist[1]:
  label[klabels==0] = LV_SURFS.ATRIAL
  label[klabels==1] = LV_SURFS.MITRAL
else:
  label[klabels==1] = LV_SURFS.ATRIAL
  label[klabels==0] = LV_SURFS.MITRAL

In [99]:
clustered = np.zeros(len(pts))
clustered[ioi] = label#+1
lvsurf.point_data["clustered"] = clustered
lvsurf.set_active_scalars("clustered")

In [100]:
plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
plotter.add_points(kcenters, color="red", point_size=300)
plotter.add_mesh(lvsurf, scalars="clustered", opacity=1.0, show_edges=False)
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [101]:
initial_guess = np.copy(endo_epi_guess)
initial_guess[clustered==LV_SURFS.ATRIAL] = LV_SURFS.ATRIAL
initial_guess[clustered==LV_SURFS.MITRAL] = LV_SURFS.MITRAL
lvsurf.point_data["initial_guess"] = initial_guess
lvsurf.set_active_scalars("initial_guess")

In [102]:
plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
plotter.add_points(kcenters, color="red", point_size=300)
plotter.add_mesh(lvsurf, scalars="initial_guess", opacity=1.0, show_edges=False)
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…

In [103]:
def est_mask(pts, initial_guess, endo_epi_guess, alpha_atr=1.5, alpha_mtr=1.5):
  atr_mask = np.where(initial_guess==LV_SURFS.ATRIAL)[0] 
  mtr_mask = np.where(initial_guess==LV_SURFS.MITRAL)[0] 
  atr_pts = pts[atr_mask]
  mtr_pts = pts[mtr_mask]
  
  c_atr = np.mean(atr_pts, axis=0)
  r_atr = np.mean(np.linalg.norm(atr_pts - c_atr, axis=1))
  c_mtr = np.mean(mtr_pts, axis=0)
  r_mtr = np.mean(np.linalg.norm(mtr_pts - c_mtr, axis=1))
  
  d_atr = np.linalg.norm(pts - c_atr, axis=1)
  d_mtr = np.linalg.norm(pts - c_mtr, axis=1)
  
  atr = np.where(d_atr <= r_atr * alpha_atr)[0]
  mtr = np.where(d_mtr <= r_mtr * alpha_mtr)[0]
  
  its = np.intersect1d(atr, mtr) # intersection
  
  # Adjust mask
  new_atr_mask = np.union1d(atr_mask, its)
  new_mtr_mask = np.union1d(mtr_mask, its)
  atr_pts = pts[new_atr_mask]
  mtr_pts = pts[new_mtr_mask]
  
  c_atr = np.mean(atr_pts, axis=0)
  r_atr = np.mean(np.linalg.norm(atr_pts - c_atr, axis=1))
  c_mtr = np.mean(mtr_pts, axis=0)
  r_mtr = np.mean(np.linalg.norm(mtr_pts - c_mtr, axis=1))
  
  d_atr = np.linalg.norm(pts - c_atr, axis=1)
  d_mtr = np.linalg.norm(pts - c_mtr, axis=1)
  
  atr = np.where(d_atr <= r_atr * alpha_atr*0.8)[0]
  mtr = np.where(d_mtr <= r_mtr * alpha_mtr*0.8)[0]
  
  its = np.intersect1d(atr, mtr) # intersection
  
  # Adjust mask
  new_atr_mask = atr
  new_mtr_mask = mtr
  atr_pts = pts[new_atr_mask]
  mtr_pts = pts[new_mtr_mask]
  atr_vecs = c_atr - atr_pts
  mtr_vecs = c_mtr - mtr_pts
  
  atr_angles = np.zeros(len(atr_vecs))
  mtr_angles = np.zeros(len(mtr_vecs))
  
  for i, (pt_normal, pt_vec) in enumerate(zip(normals[new_atr_mask], atr_vecs)):
    atr_angles[i] = angle_between(pt_vec, pt_normal, check_orientation=False)
    
  for i, (pt_normal, pt_vec) in enumerate(zip(normals[new_mtr_mask], mtr_vecs)):
    mtr_angles[i] = angle_between(pt_vec, pt_normal, check_orientation=False)
  
  # print(mtr_vecs.shape, mtr_angles.shape, new_mtr_mask.shape, mtr.shape)
  atr = new_atr_mask[np.where(atr_angles <= np.radians(89))[0]]
  mtr = new_mtr_mask[np.where(mtr_angles <= np.radians(89))[0]]
  # ep1 = new_mtr_mask[np.where(mtr_angles > np.radians(89))[0]]
  
  
  new_guess = np.copy(endo_epi_guess)
  # new_guess[ep1] = 6
  new_guess[atr] = LV_SURFS.ATRIAL
  new_guess[mtr] = LV_SURFS.MITRAL
  new_guess[its] = LV_SURFS.INTERCECTION
  
  
  
  return new_guess, (c_atr, c_mtr)

In [104]:
mask, (c_atr, c_mtr) = est_mask(pts, initial_guess, endo_epi_guess, 
                                alpha_atr=1.7, alpha_mtr=1.5)
lvsurf.point_data["mask"] = mask
lvsurf.set_active_scalars("mask")

In [105]:
plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
# plotter.add_points(kcenters, color="beige", point_size=300)
plotter.add_points(c_atr, color="red", point_size=300)
plotter.add_points(c_mtr, color="red", point_size=300)
plotter.add_mesh(lvsurf, scalars="mask", cmap="jet",
                 opacity=1.0, show_edges=False)
plotter.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#fefefe', inten…